In [1]:
import csv
import os
import random
import sklearn.model_selection

from shutil import copyfile

In [2]:
TRAIN_TEST_CSV_PATH = '../C1-P1_Train Dev_fixed/train.csv'
TRAIN_CSV_PATH = '../C1-P1_Train Dev_fixed/train_split.csv'
VALID_CSV_PATH = '../C1-P1_Train Dev_fixed/dev.csv'

ORIGINAL_TRAIN_TEST_DATA_PATH = '../C1-P1_Train Dev_fixed/C1-P1_Train/' 
# ORIGINAL_TRAIN_DATA_PATH = '../C1-P1_Train Dev_fixed/C1-P1_Train/' 
ORIGINAL_VALID_DATA_PATH = '../C1-P1_Train Dev_fixed/C1-P1_Dev/' 


TRAIN_DATA_PATH = '../data/train'
VALID_DATA_PATH = '../data/valid'
TEST_DATA_PATH = '../data/test'

NUM_TEST_DATASET = 100

image_size = 224

# Split data from train dataset. get last 100 records as test dataset

In [3]:
new_train_data = []
new_test_data = []
all_data = []

TEST_CSV_PATH = '../C1-P1_Train Dev_fixed/test_split.csv'

with open(TRAIN_TEST_CSV_PATH) as csv_file:

    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            line_count += 1
            continue  #header
        all_data.append(row)
        
        line_count += 1
        
indices = list(range(len(all_data)))

random.shuffle(indices)
training_dataset, test_dataset = sklearn.model_selection.train_test_split(indices, train_size=len(all_data)-NUM_TEST_DATASET, test_size=NUM_TEST_DATASET)

for idx in training_dataset:
    new_train_data.append(all_data[idx])
for idx in test_dataset:
    new_test_data.append(all_data[idx])  

with open(TRAIN_CSV_PATH, 'w') as f:

    writer = csv.writer(f)
    
    writer.writerow(['image_id','label'])
    for row in new_train_data:
        writer.writerow(row)
        
with open(TEST_CSV_PATH, 'w') as f:

    writer = csv.writer(f)
    
    writer.writerow(['image_id','label'])
    for row in new_test_data:
        writer.writerow(row)

In [4]:
# parse csv file and make the folder structure for pytorch test dataset
def prepare_file_structure_for_pytoch(csv_path, src_data_path, dst_data_path):
    with open(csv_path) as csv_file:

        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                line_count += 1
                continue  #header

            src_path = os.path.join(src_data_path, row[0])
            dest_path = os.path.join(dst_data_path, row[1], row[0])
            dest_folder_path = os.path.join(dst_data_path, row[1])
            if not os.path.exists(dest_folder_path):
                os.makedirs(dest_folder_path)
            if not os.path.isfile(dest_path):
                copyfile(src_path, dest_path)
        
            line_count += 1

In [5]:
!rm -rf ../data/train
!rm -rf ../data/test
!rm -rf ../data/valid
# make file structure for training dataset
prepare_file_structure_for_pytoch(TRAIN_CSV_PATH, ORIGINAL_TRAIN_TEST_DATA_PATH, TRAIN_DATA_PATH)
    
# make file structure for validation dataset
prepare_file_structure_for_pytoch(VALID_CSV_PATH, ORIGINAL_VALID_DATA_PATH, VALID_DATA_PATH)

# make file structure for test dataset
prepare_file_structure_for_pytoch(TEST_CSV_PATH, ORIGINAL_TRAIN_TEST_DATA_PATH, TEST_DATA_PATH)